## ¿Cómo Amazon DynamoDB utiliza AWS KMS?

El cifrado disponible en DynamoDB ayuda a configurar una capa adicional de seguridad para que los datos transmitidos no sean visibles para terceros. Admite tanto el cifrado del lado del cliente como el cifrado en reposo. DynamoDB implementa su cifrado en:

* Datos en reposo
* Datos en tránsito

Los datos se cifrarán y firmarán antes de enviarse a AWS DynamoDB. Mientras se almacenan, los datos pasarán por un proceso de cifrado, proporcionando así una protección de extremo a extremo contra cambios no autorizados en los datos.

DynamoDB se integra con AWS Key Management Service (AWS KMS) para admitir una característica de cifrado del lado del servidor de cifrado en reposo. Con el cifrado en reposo, DynamoDB cifra de forma transparente todos los datos de los clientes en una tabla de DynamoDB, incluida su clave principal y los índices secundarios locales y globales, siempre que la tabla se almacene en el disco. (Si la tabla tiene una clave de ordenación, algunas de las claves de ordenación que marcan los límites del intervalo se almacenan en texto no cifrado en los metadatos de la tabla). Cuando obtiene acceso a la tabla, DynamoDB descifra los datos de la tabla de forma transparente. No es necesario que cambie sus aplicaciones para utilizar o administrar tablas cifradas.

El cifrado en reposo protege también las secuencias de DynamoDB, las tablas globales y las copias de seguridad siempre que estos objetos se guardan en un soporte duradero. Las instrucciones acerca de las tablas de este tema se aplican también a estos objetos.

Todas las tablas de DynamoDB están cifradas. No existe la opción de habilitar o deshabilitar el cifrado para tablas nuevas o existentes. De forma predeterminada, todas las tablas están cifradas en una Clave propiedad de AWS en la cuenta de servicio de DynamoDB. Sin embargo, puede seleccionar una opción para cifrar algunas o todas las tablas con una clave administrada por el cliente o la Clave administrada por AWS para DynamoDB en su cuenta.

In [1]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import base64
from spdynamodb import DynamoTable
from datetime import datetime
import json
import time
from decimal import Decimal

In [2]:
dt=DynamoTable()
try:
    dt.select_table('Books')
    print(dt)
except:
    dt.create_table(
        table_name='Books',
        partition_key='Author',
        partition_key_type='S',
        sort_key='Title',
        sort_key_type='S',
    )

Table created successfully!


In [5]:
# Create a Books Dataframe

df = pd.DataFrame({
    'Title': ['The Great Gatsby','To Kill a Mockingbird','1984', 'Pride and Prejudice', 'The Odyssey'],
    'Author': ['F. Scott Fitzgerald','Harper Lee','George Orwell', 'Jane Austen', 'Homer'],
    'Length': [180, 281, 328, 226, 374],
    'Published': [1925, 1960, 1949, 1813, 1922],
    'Publisher': ['Charles Scribner\'s Sons','J.B. Lippincott & Co.','Secker & Warburg', 'Penguin','Scribner']
})
dt.batch_pandas(df)

Al crear una tabla nueva, puede elegir uno de los siguientes tipos de AWS KMS key para cifrarla. Puede cambiar entre estos tipos de claves en cualquier momento.

* **Clave propiedad de AWS**: tipo de cifrado predeterminado. La clave es propiedad de DynamoDB (sin cargo adicional).

* **Clave administrada por AWS**: la clave se almacena en la cuenta y la administra AWS KMS (se aplican los cargos de AWS KMS).

* **Customer managed key** (Clave administrada por el cliente): la clave se almacena en la cuenta y usted la crea, posee y administra. Usted controla plenamente la clave KMS (se aplican los cargos de AWS KMS).

In [8]:
# Create an STS client
sts = boto3.client('sts')

identity = sts.get_caller_identity()
account_id = identity['Account']

In [9]:
policy = {
    "Id": "protected-cmk-policy",
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "Enable IAM User Permissions",
            "Effect": "Allow",
            "Principal": {
                "AWS": f"arn:aws:iam::{account_id}:root"
            },
            "Action": "kms:*",
            "Resource": "*"
        },
        {
            "Sid": "Allow access for Key Administrators",
            "Effect": "Allow",
            "Principal": {
                "AWS": f"arn:aws:iam::{account_id}:user/cloud_user"
            },
            "Action": [
                "kms:Create*",
                "kms:Describe*",
                "kms:Enable*",
                "kms:List*",
                "kms:Put*",
                "kms:Update*",
                "kms:Revoke*",
                "kms:Disable*",
                "kms:Get*",
                "kms:Delete*",
                "kms:TagResource",
                "kms:UntagResource",
                "kms:ScheduleKeyDeletion",
                "kms:CancelKeyDeletion"
            ],
            "Resource": "*"
        },
        {
            "Sid": "Allow use of the key",
            "Effect": "Allow",
            "Principal": {
                "AWS": f"arn:aws:iam::{account_id}:user/cloud_user"
            },
            "Action": [
                "kms:Encrypt",
                "kms:Decrypt",
                "kms:ReEncrypt*",
                "kms:GenerateDataKey*",
                "kms:DescribeKey"
            ],
            "Resource": "*"
        },
        {
            "Sid": "Allow attachment of persistent resources",
            "Effect": "Allow",
            "Principal": {
                "AWS": f"arn:aws:iam::{account_id}:user/cloud_user"
            },
            "Action": [
                "kms:CreateGrant",
                "kms:ListGrants",
                "kms:RevokeGrant"
            ],
            "Resource": "*",
            "Condition": {
                "Bool": {
                    "kms:GrantIsForAWSResource": "true"
                }
            }
        }
    ]
}

In [10]:
kms = boto3.client('kms')
try:
   response = kms.create_key(
       Policy=json.dumps(policy),
       Description='Customer Master Key for DynamoDB Data Encryption',
   )
except ClientError as e:
   print(e)
else:
   key_id = response["KeyMetadata"]["KeyId"]
   key_arn = response["KeyMetadata"]["Arn"]
   print("Key ID:", key_id)
   print("Key ARN:", key_arn)

Key ID: f4de6897-c3b5-4acd-92fa-dd08e7f04ea3
Key ARN: arn:aws:kms:us-east-1:175189267104:key/f4de6897-c3b5-4acd-92fa-dd08e7f04ea3


In [30]:
alias = 'alias/DynamoDBTableDataCMK'
try:
    response = kms.create_alias(
        AliasName=alias,
        TargetKeyId=key_id
    )
    print(response)
except Exception as e:
    print(e)

{'ResponseMetadata': {'RequestId': '88d24ee1-21b0-43f6-9651-389359c6a2f1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '88d24ee1-21b0-43f6-9651-389359c6a2f1', 'cache-control': 'no-cache, no-store, must-revalidate, private', 'expires': '0', 'pragma': 'no-cache', 'date': 'Tue, 05 Dec 2023 00:36:02 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [39]:
dt.table.update(
    SSESpecification={
        "Enabled": True,
        "SSEType": "KMS",
        "KMSMasterKeyId": key_id
    }
)

dynamodb.Table(name='Books')

In [40]:
dt.table.sse_description

{'Status': 'UPDATING',
 'SSEType': 'KMS',
 'KMSMasterKeyArn': 'arn:aws:kms:us-east-1:827456662682:key/6aefe290-028d-45d0-ae5d-c20a2bf7102e'}

In [140]:
class KeyClass:
    def __init__(self, kms_client):
        self.kms_client = kms_client

    def encrypt(self, key_id, text):
        """
        Encrypts text by using the specified key.

        :param key_id: The ARN or ID of the key to use for encryption.
        :return: The encrypted version of the text.
        """
        try:
            cipher_text = self.kms_client.encrypt(
                KeyId=key_id, Plaintext=text.encode()
            )["CiphertextBlob"]
        except ClientError as err:
            print(
                f"Couldn't encrypt text. Here's why: {err.response['Error']['Message']}"
            )
        else:
            return cipher_text


    def decrypt(self, key_id, cipher_text):
        """
        Decrypts text previously encrypted with a key.

        :param key_id: The ARN or ID of the key used to decrypt the data.
        :param cipher_text: The encrypted text to decrypt.
        """
        try:
            text = self.kms_client.decrypt(
                KeyId=key_id, CiphertextBlob=cipher_text
            )["Plaintext"]
        except ClientError as err:
            print(
                f"Couldn't decrypt your ciphertext. Here's why: {err.response['Error']['Message']}"
            )
        else:
            return text.decode()

In [143]:
kms = boto3.client('kms')
key_class = KeyClass(kms)
encrpyt_text = key_class.encrypt(key_id, "Some text")
print(f"Your ciphertext is: {encrpyt_text}")

Your ciphertext is: b'\x01\x02\x02\x00xs\xd5No\x1d:#\xbeP4kU~G\x8d\xea\xe1c",\xea\x8e\xaatX\xe9\xa7d\xf3\xa3\xa4\x93\x01\xe3\xe5\xb5*\xf6\xb0\x99\x94C\x8d)G*D\x87\xfb\x00\x00\x00g0e\x06\t*\x86H\x86\xf7\r\x01\x07\x06\xa0X0V\x02\x01\x000Q\x06\t*\x86H\x86\xf7\r\x01\x07\x010\x1e\x06\t`\x86H\x01e\x03\x04\x01.0\x11\x04\x0c\x0e\x03:\xf7$,(\x83\xc9_\x07A\x02\x01\x10\x80$\xd8l\x95\x1eJN\x9e@\xba.c7y\x1adjY\xf5d\x17\x8a\xcb\r\xce\xece@[\xa8\x11\xf1\x90\xdf(\x1c0'


In [142]:
decrypt_text = key_class.decrypt(key_id, encrpy_text)
print(f"Your plaintext is: {decrypt_text}")

Your plaintext is: Some text
